In [ ]:
import os

In [ ]:
DATA_DIR = 'trec07p/data/'
LABELS_FILE = 'trec07p/full/index'
TRAINING_SET_RATIO = 0.7

In [ ]:
labels = {}
spam_words = set()
ham_words = set()

In [ ]:
# Read the labels
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

In [ ]:
# Split corpus into train and test sets
filelist = os.listdir(DATA_DIR)
X_train = filelist[:int(len(filelist)*TRAINING_SET_RATIO)]
X_test = filelist[int(len(filelist)*TRAINING_SET_RATIO):]

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
import email
import re

TRUSTED_DOMAINS = {'gmail.com', 'yahoo.com', 'outlook.com', 'company.com'}  # دامنه‌های معتبر
SPAM_DOMAINS = {'free-mail.com', 'spam4u.net', 'cheap-mail.pro'}  # دامنه‌های اسپم
SUSPICIOUS_SERVICES = {'mail.ru', 'yandex.com'}  # سرویس‌های با ریسک متوسط

In [ ]:
# 2. تابع استخراج اطلاعات فرستنده (نسخه اصلاح شده)
def extract_sender_info(email_path):
    try:
        with open(email_path, 'rb') as f:  # باز کردن فایل به صورت باینری
            msg = email.message_from_binary_file(f)  # استفاده از message_from_binary_file
            if msg is None:
                return '', ''  # در صورت شکست در خواندن ایمیل

        sender = msg.get('From', '')
        domain = ''
        service = ''

        # استخراج دامنه از آدرس ایمیل
        if isinstance(sender, str):  # اطمینان از رشته بودن sender
            email_match = re.search(r'@([a-zA-Z0-9.-]+)', sender)
            if email_match:
                domain = email_match.group(1).lower()

        # استخراج سرویس از Received headers
        received_headers = msg.get_all('Received', [])
        for header in received_headers:
            if isinstance(header, str):  # فقط هدرهای رشته‌ای پردازش شوند
                service_match = re.search(r'with (SMTP|ESMTP|HTTP|IMAP)', header)
                if service_match:
                    service = service_match.group(1)
                    break

        return domain, service
    except Exception as e:
        print(f"Error in extract_sender_info for {email_path}: {str(e)}")
        return '', ''  # در صورت خطا، مقادیر خالی برگردانده شود


In [ ]:

# 3. بارگذاری لیبل‌ها
labels = {}
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        if len(parts) >= 2:
            label, path = parts[0], parts[1]
            labels[os.path.basename(path)] = 1 if label.lower() == 'ham' else 0

# 4. تحلیل ایمیل‌ها
results = []
for filename in list(labels.keys()):  # ایجاد لیست از کلیدها برای جلوگیری از تغییر اندازه
    try:
        path = os.path.join(DATA_DIR, filename)
        if not os.path.exists(path):
            continue

        domain, service = extract_sender_info(path)
        true_label = labels[filename]

        # اعتبارسنجی (متریک اصلی)
        if domain in SPAM_DOMAINS:
            predicted_label = 0  # اسپم
        elif domain in TRUSTED_DOMAINS:
            predicted_label = 1  # هم
        elif domain in SUSPICIOUS_SERVICES:
            predicted_label = 0  # اسپم (با احتمال کمتر)
        else:
            predicted_label = 0  # ناشناخته = اسپم در نظر گرفته می‌شود

        results.append((true_label, predicted_label))
    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")
        continue


In [ ]:
# 5. محاسبه معیارهای ارزیابی
if results:  # فقط اگر داده‌ای برای تحلیل وجود داشته باشد
    tp = sum(1 for true, pred in results if true == 0 and pred == 0)
    tn = sum(1 for true, pred in results if true == 1 and pred == 1)
    fp = sum(1 for true, pred in results if true == 1 and pred == 0)
    fn = sum(1 for true, pred in results if true == 0 and pred == 1)

    accuracy = (tp + tn) / len(results)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # 6. نمایش نتایج
    display(HTML('<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join('<td>{}</td>'.format(
            '</td><td>'.join(str(_) for _ in row))
                         for row in conf_matrix))))

    count = tn + tp + fn + fp
    percent_matrix = [["{:.1%}".format(tn/count), "{:.1%}".format(fp/count)],
                      ["{:.1%}".format(fn/count), "{:.1%}".format(tp/count)]]
    display(HTML('<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join('<td>{}</td>'.format(
            '</td><td>'.join(str(_) for _ in row))
                         for row in percent_matrix))))

    print("Classification accuracy: {}".format("{:.1%}".format((tp+tn)/count)))
    print("Precision (spam): {}".format("{:.1%}".format(tp/(tp+fp))))
    print("Recall (spam): {}".format("{:.1%}".format(tp/(tp+fn))))
    print("F1-score (spam): {}".format("{:.1%}".format(2*tp/(2*tp + fp + fn))))

else:
    print("هیچ داده‌ای برای تحلیل یافت نشد.")